In [1]:
import numpy as np
import torch as th


from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ray.init(num_cpus=160, ignore_reinit_error=True)

2023-08-25 10:58:50,373	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.11.3
Ray version:,2.6.2
Dashboard:,http://127.0.0.1:8266


In [3]:
insmp = Backtester(20190103, 20210101)
otsmp = Backtester(20210101, 20211231)

num of factors:124
num of factors:124


In [4]:
factors = insmp.calc_factor()

calc_factor took 239 seconds


In [5]:
factors2 = otsmp.calc_factor()

calc_factor took 140 seconds


In [7]:
df_ic = insmp.calc_ic()
df_ic2 = otsmp.calc_ic()

calc_ic took 584 seconds
calc_ic took 285 seconds


In [8]:
insmp.df_ic.query("y1>.01 and y2>.015 and y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.042246,0.046690,0.060397
20230813143935_satd_lexpr08_lopt34_10_0322,0.019739,0.023186,0.032060
20230813143936_satd_lexpr08_lopt34_10_0714,0.020054,0.023814,0.034957
20230813145451_satd_lexpr08_lopt34_10_0856,0.045665,0.051074,0.071974
20230813145458_satd_lexpr08_lopt34_10_0262,0.039762,0.043808,0.061600
...,...,...,...
20230825003101_satd_lexpr10_lopt50_10_3581,0.048912,0.053101,0.063885
20230825090920_satd_lexpr10_lopt50_10_9548,0.014690,0.017996,0.025416
20230825090924_satd_lexpr10_lopt50_10_9750,0.013659,0.015380,0.028290
20230825090927_satd_lexpr10_lopt50_10_5052,0.028705,0.031422,0.040526


In [9]:
otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.029942,0.032678,0.039297
20230813143935_satd_lexpr08_lopt34_10_0322,0.011797,0.013961,0.012473
20230813145451_satd_lexpr08_lopt34_10_0856,0.026510,0.029783,0.036930
20230813145458_satd_lexpr08_lopt34_10_0262,0.023295,0.025203,0.033990
20230813145501_satd_lexpr08_lopt34_10_0726,0.029467,0.029454,0.030696
...,...,...,...
20230825003035_satd_lexpr10_lopt50_10_6173,0.019370,0.022707,0.023508
20230825003047_satd_lexpr10_lopt50_10_5577,0.025244,0.027922,0.030913
20230825003101_satd_lexpr10_lopt50_10_3581,0.026536,0.029207,0.036954
20230825090927_satd_lexpr10_lopt50_10_5052,0.015517,0.017328,0.017932


In [10]:
selected_idx = otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02").index

In [11]:
feature_dict = dict(zip(otsmp.factor_names,otsmp.factors))

In [15]:
selected_sigs = [feature_dict[i] for i in selected_idx]

In [17]:
def calc_sig_corr(sigs):
    n = len(sigs)
    df_corr = np.ones((n,n))
    corrs = []
    for i in range(n):
        for j in range(i+1,n):
            ic = torch.nanmean(batch_pearsonr(sigs[i],sigs[j],))
            df_corr[i,j] = ic
            df_corr[j,i] = ic
    return df_corr
            

In [19]:
df_corr = calc_sig_corr(selected_sigs)

In [21]:
df_corr.shape

(87, 87)

In [23]:
df_corr = pd.DataFrame(df_corr)

In [26]:
sigs = remove_high_corr_factors(df_corr.values)

In [28]:
len(sigs)

64

In [36]:
final = [selected_idx[idx] for idx in sigs]

In [47]:

# 设置显示最大列数
pd.set_option('display.max_columns', None)

# 设置显示最大行数
pd.set_option('display.max_rows', None)
finaldf =  otsmp.df_ic.loc[final]

In [49]:
print(finaldf)

                                                  y1        y2        y5
20230811171800_satd_lexpr08_lopt34_10_0042  0.029942  0.032678  0.039297
20230813143935_satd_lexpr08_lopt34_10_0322  0.011797  0.013961  0.012473
20230813145451_satd_lexpr08_lopt34_10_0856  0.026510  0.029783  0.036930
20230813145501_satd_lexpr08_lopt34_10_0726  0.029467  0.029454  0.030696
20230814092613_satd_lexpr08_lopt34_10_0788  0.030076  0.031085  0.033477
20230814092614_satd_lexpr08_lopt34_10_0177  0.010392  0.011457  0.016512
20230814092615_satd_lexpr08_lopt34_10_0018  0.022174  0.023574  0.024581
20230814092615_satd_lexpr08_lopt34_10_0143  0.022328  0.025780  0.027584
20230814095857_satd_lexpr08_lopt34_10_0578  0.028826  0.031991  0.031900
20230814102408_satd_lexpr15_lopt34_10_0319  0.026077  0.027266  0.035737
20230814175110_satd_lexpr15_lopt34_10_0269  0.025901  0.027344  0.033700
20230814175115_satd_lexpr15_lopt34_10_0852  0.035336  0.036327  0.041362
20230815095228_satd_lexpr10_lopt34_10_0762  0.02167

In [50]:
finaldf.describe()

,y1,y2,y5
count,64.000000,64.000000,64.000000
mean,0.023043,0.024733,0.027598
std,0.006447,0.006554,0.008456
min,0.010208,0.011457,0.011241
25%,0.018774,0.020391,0.021407
50%,0.023624,0.025460,0.028360
75%,0.026870,0.029210,0.033970
max,0.036442,0.038170,0.044213


In [68]:
fs  = selected_sigs
y1d = otsmp.calter.ret1d
y2d = otsmp.calter.ret2d
y5d = otsmp.calter.ret5d

In [69]:
q1d = [batch_topk(fs[i],y1d) for i in range(len(fs))]
q2d = [batch_topk(fs[i],y2d) for i in range(len(fs))]
q5d = [batch_topk(fs[i],y5d) for i in range(len(fs))]

In [75]:
q1d = [q1d[idx] for idx in sigs]
q2d = [q2d[idx] for idx in sigs]
q5d = [q5d[idx] for idx  in sigs]

In [76]:
finaldf['q1d'] = q1d
finaldf['q2d'] = q2d
finaldf['q5d'] = q5d

In [77]:
finaldf

,y1,y2,y5,q1d,q2d,q5d
20230811171800_satd_lexpr08_lopt34_10_0042,0.029942,0.032678,0.039297,"(0.00025, 0.00023)","(0.00051, 0.00037)","(0.00126, 0.00175)"
20230813143935_satd_lexpr08_lopt34_10_0322,0.011797,0.013961,0.012473,"(0.00016, 0.00016)","(0.0004, 0.00035)","(0.00082, 0.00105)"
20230813145451_satd_lexpr08_lopt34_10_0856,0.026510,0.029783,0.036930,"(0.00028, -0.00033)","(0.00053, -0.00037)","(0.00159, 0.00044)"
20230813145501_satd_lexpr08_lopt34_10_0726,0.029467,0.029454,0.030696,"(0.00024, 3e-05)","(0.00043, -6e-05)","(0.00045, -0.00064)"
20230814092613_satd_lexpr08_lopt34_10_0788,0.030076,0.031085,0.033477,"(0.00017, -0.00019)","(0.00038, -0.00038)","(0.00082, -0.0006)"
20230814092614_satd_lexpr08_lopt34_10_0177,0.010392,0.011457,0.016512,"(-0.00035, -0.00059)","(-0.00046, -0.001)","(0.00017, -0.0003)"
20230814092615_satd_lexpr08_lopt34_10_0018,0.022174,0.023574,0.024581,"(8e-05, -6e-05)","(0.00018, -0.00019)","(0.00071, 0.00156)"
20230814092615_satd_lexpr08_lopt34_10_0143,0.022328,0.025780,0.027584,"(0.00025, -0.00101)","(0.00059, -0.00101)","(0.00133, -0.0004)"
20230814095857_satd_lexpr08_lopt34_10_0578,0.028826,0.031991,0.031900,"(0.00071, 0.00039)","(0.00117, 0.00047)","(0.00186, 0.00077)"
20230814102408_satd_lexpr15_lopt34_10_0319,0.026077,0.027266,0.035737,"(-3e-05, -0.00114)","(-1e-05, -0.00143)","(0.00139, -0.00011)"


In [78]:
finaldf.to_csv('result.csv')

In [84]:
df_corr.iloc[sigs,sigs].to_csv('df_corr.csv')

In [90]:
df_corr.replace(1,0).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86
count,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000
mean,0.394188,0.345408,0.526377,0.532650,0.330428,0.446560,0.244102,0.488133,0.360277,0.383190,0.297265,0.521671,0.522205,0.554274,0.531588,0.373864,0.502993,0.385635,0.543003,0.497303,0.542268,0.566872,0.425137,0.337642,0.356015,0.525475,0.499933,0.511696,0.139952,0.535619,0.565128,0.443213,0.339075,0.400773,0.560777,0.527214,0.430647,0.575319,0.192890,0.404443,0.525875,0.369063,0.561627,0.567584,0.337253,0.360836,0.530592,0.420536,0.460509,0.554970,0.359956,0.474533,0.422131,0.295767,0.399494,0.386340,0.421914,0.449009,0.475146,0.361519,0.321051,0.347418,0.414613,0.461986,0.558130,0.438660,0.383981,0.526480,0.236425,0.387305,0.196517,0.442558,0.363438,0.188999,0.558120,0.206444,0.564022,0.397621,0.342509,0.156985,0.466907,0.542708,0.434134,0.571188,0.405957,0.294827,0.499641
std,0.187749,0.160283,0.163322,0.185872,0.108957,0.142278,0.182683,0.153282,0.156905,0.110081,0.140148,0.208607,0.175226,0.201293,0.163111,0.171001,0.180410,0.140163,0.189479,0.166507,0.183482,0.171473,0.147205,0.129944,0.139530,0.191637,0.152441,0.174965,0.098468,0.185804,0.174518,0.141262,0.190485,0.148106,0.187395,0.167912,0.155539,0.186112,0.071783,0.179177,0.211567,0.139095,0.174418,0.182951,0.140754,0.124628,0.184504,0.147604,0.132886,0.192931,0.163811,0.144750,0.145000,0.114885,0.138109,0.153058,0.144977,0.173110,0.151647,0.151004,0.172778,0.136186,0.145696,0.167373,0.169063,0.193975,0.133913,0.195201,0.093078,0.120864,0.090935,0.134893,0.158208,0.094900,0.207742,0.086423,0.201577,0.194913,0.097857,0.064717,0.161616,0.188563,0.138409,0.194458,0.142983,0.133607,0.164422
min,0.000000,-0.076892,0.000000,0.000000,0.000000,0.000000,-0.113299,0.000000,-0.148033,0.000000,-0.121817,0.000000,0.000000,0.000000,0.000000,-0.133113,-0.004620,-0.076755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.148033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.096922,0.000000,0.000000,-0.026876,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.045142,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004586,0.000000,0.000000,0.000000,-0.090778,-0.113299,0.000000,0.000000,0.000000,0.000000,-0.155678,0.000000,0.000000,0.000000,-0.026876,0.000000,-0.010749,-0.155678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.060776,0.000000
25%,0.232072,0.258506,0.422853,0.393248,0.260701,0.337540,0.095457,0.382025,0.263554,0.306662,0.182743,0.365552,0.398277,0.409742,0.433271,0.264041,0.385950,0.323907,0.422629,0.365493,0.385977,0.464709,0.347063,0.263609,0.289999,0.378534,0.470982,0.395003,0.079966,0.401684,0.442061,0.367966,0.183388,0.325644,0.440331,0.416717,0.365326,0.481076,0.148313,0.316139,0.353470,0.309597,0.452138,0.433844,0.230967,0.300880,0.385797,0.312288,0.361237,0.402401,0.268545,0.385469,0.352923,0.223502,0